# Compatification of feature space
Analysing decison boundaries is not an easy task, especially given the fact that the feature space is non compact.

On compact spaces it is easier to work, as they a re close and bounded (Heine-Borel). 

We propose here a method to compactifiy the feature space $\mathbb R^n$ to the projective space $\mathbb RP^n$.

The decision boundary, gets therefore sampled in each chart of $\mathbb RP^n$ uniformly. When charts are put together, the resulting point cloud (defined abstractly via a dissimilarity matrix `d_final`), can be used to compute the topology of the *compactified* decision boundary.

We believe that the topology so obtained can furthe be exploited for regularisation purposes.

In [1]:
%reload_ext autoreload
%autoreload 2

# deep learning
import torch
from torch.optim import Adam, SGD
import numpy as np
from torch import nn
from gdeep.models import FFNet
from gdeep.data import TorchDataLoader
from gdeep.pipeline import Pipeline
from torch import autograd  

# plot
import plotly.express as px
import pandas as pd
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

# ML
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_blobs
from sklearn.metrics import pairwise_distances

# TDA
from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram


# Build datatset

We want to test our method on a 3D dataset made of 2 separate blob. We expect that the neural network decision boundary looks like and hyperplane in $\mathbb R^3$.

After compactification, we would expect to find $\mathbb RP^2$ as final result.

In [2]:
dl = TorchDataLoader(name="Blobs")
dl_tr, dl_ts = dl.build_dataloader(batch_size=1)

In [3]:
next(iter(dl_tr))

[tensor([[-3.7321,  9.5506,  3.7252]]), tensor([0])]

In [4]:
# train NN
model = FFNet(arch=[3,3])
print(model)
pipe = Pipeline(model, (dl_tr, dl_ts), nn.CrossEntropyLoss(), writer)
pipe.train(SGD, 5, batch_size=1, lr=0.01)

FFNet(
  (layer0): Linear(in_features=3, out_features=3, bias=True)
)
TOTAL EPOCHS  5
Epoch 1
-------------------------------
Training loss: 0.012591  [160/160]
Time taken for this epoch: 0s
Validation results: 
 Accuracy: 13.5%,                 Avg loss: 0.073809 

Epoch 2
-------------------------------
Training loss: 1.098612  [160/160]
Time taken for this epoch: 0s
Validation results: 
 Accuracy: 13.5%,                 Avg loss: 0.073017 

Epoch 3
-------------------------------
Training loss: 1.098612  [160/160]
Time taken for this epoch: 0s
Validation results: 
 Accuracy: 13.5%,                 Avg loss: 0.072661 

Epoch 4
-------------------------------
Training loss: 1.098612  [160/160]
Time taken for this epoch: 0s
Validation results: 
 Accuracy: 13.5%,                 Avg loss: 0.072489 

Epoch 5
-------------------------------
Training loss: 0.003249  [160/160]
Time taken for this epoch: 0s
Validation results: 
 Accuracy: 13.5%,                 Avg loss: 0.072322 

Test resu

In [5]:
from gdeep.visualisation import Visualiser

vs = Visualiser(pipe)
vs.plot_data_model()
db, d_final, _ = vs.plot_decision_boundary(True)


AttributeError: module 'tensorflow._api.v2.io.gfile' has no attribute 'get_filesystem'

# Topology
We chec with Giotto-tda that the topology of the decison boundary is indeed that one of $\mathbb RP^2$, as expected.

In [ ]:
# check topology from d_final

vr = VietorisRipsPersistence(collapse_edges=True, max_edge_length=1,
                             metric="precomputed", n_jobs=-1, 
                             homology_dimensions=(0,1,2))
diag = vr.fit_transform([d_final])

plot_diagram(diag[0])